In [1]:
# 머신러닝 알고리즘 적용

from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

train = pd.read_csv(r'C:\Users\ssjy1\kaggle\datasets\datasets_titanic_train.csv')
test = pd.read_csv(r'C:\Users\ssjy1\kaggle\datasets\datasets_titanic_test.csv')

def feature_engineering(df):
    # Sex
    df['Sex'] = df['Sex'].map({'female': 0, 'male': 1})

    # Embarked
    df.Embarked.fillna('S', inplace=True)
    df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

    # Title - 이름으로 호칭 추출.
    df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.')  # 공백으로 시작하고 .으로 끝나는 문자열 추출
    df['Title'] = df['Title'].replace(['Capt', 'Col', 'Countess', 'Don','Dona', 'Dr', 'Jonkheer', 'Lady','Major', 'Rev', 'Sir'], 'Other')
    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')
    df['Title'] = df['Title'].replace('Ms', 'Miss')
    df['Title'] = df['Title'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs':3, 'Other':4})

    # Age - 8개 구간으로 분리하여 카테고리화. 결측치는 호칭의 평균값으로 채움.
    meanAge = df[['Title', 'Age']].groupby(['Title']).mean()  # 그룹별 나이의 평균치
    for index, row in meanAge.iterrows():
        nullIndex = df[(df.Title == index) & (df.Age.isnull())].index
        df.loc[nullIndex, 'Age'] = row[0]  # null data에 나이 평균치 임의 입력

    df['AgeCategory'] = pd.qcut(df.Age, 8, labels=range(1, 9))  # 동일 갯수로 나누기 범주 만들기
    df.AgeCategory = df.AgeCategory.astype(int)

    # Cabin - 방번호의 첫 글자만 따서 카테고리화. 결측치는 N으로 채움.
    df.Cabin.fillna('N', inplace=True)
    df["CabinCategory"] = df["Cabin"].str.slice(start=0, stop=1)
    df["CabinCategory"] = df['CabinCategory'].map({ "N": 0, "C": 1, "B": 2, "D": 3, "E": 4, "A": 5, "F": 6, "G": 7, "T": 8 })

    # Fare - 운임을 8개의 구간으로 분리하여 카테고리화. 결측치는 0으로 채움.
    df.Fare.fillna(0, inplace=True)
    df['FareCategory'] = pd.qcut(df.Fare, 8, labels=range(1, 9))
    df.FareCategory = df.FareCategory.astype(int)

    # SibSp, Parch - 가족으로 합치고, 혼자 탑승한 고객 정보 추출.
    df['Family'] = df['SibSp'] + df['Parch'] + 1  # 가족 수 + 본인
    df.loc[df["Family"] > 4, "Family"] = 5  # 5명 이상은 전부 5

    df['IsAlone'] = 1
    df.loc[df['Family'] > 1, 'IsAlone'] = 0  # 한 명 이상은 isAlone이 false

    # Ticket - 티켓 번호의 첫 번째 번호만 추출하여 카테고리화.
    df['TicketCategory'] = df.Ticket.str.split()
    df['TicketCategory'] = [i[-1][0] for i in df['TicketCategory']]
    df['TicketCategory'] = df['TicketCategory'].replace(['8', '9', 'L'], '8')
    df['TicketCategory'] = pd.factorize(df['TicketCategory'])[0] + 1

    # axis=1 : 컬럼 삭제. axis=0 : 로우 삭제.
    # inplace=True : drop한 후의 데이터프레임으로 기존 데이터프레임을 대체.
    df.drop(['PassengerId', 'Ticket', 'Cabin', 'Fare', 'Name', 'Age', 'SibSp', 'Parch'], axis=1, inplace=True)

    return df

train = feature_engineering(train)
test = feature_engineering(test)



# ========== machine learning - 랜덤 포레스트 검증 1 ==========
#  Pclass, Sex, Embarked, Title, AgeCategory, CabinCategory, FareCategory, Family, IsAlone, TicketCategory 
data = train.drop('Survived', axis=1).values
# Survived
target = train['Survived'].values

# test_size: 분리 비율 설정. 
# stratify: 분리 기준이 될 데이터 
# random_state: 랜덤 seed
x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size=0.4, stratify=target, random_state=0)

# n_estimators=50 : 트리 갯수
rf = RandomForestClassifier(n_estimators=50, criterion="entropy", max_depth=5, oob_score=True, random_state=10)
rf.fit(x_train, y_train)
prediction = rf.predict(x_valid)

length = y_valid.shape[0]
accuracy = accuracy_score(prediction, y_valid)
print(f'총 {length}명 중 {accuracy * 100:.3f}% 정확도로 생존을 맞춤')
print('\n')


# ========== machine learning - 알고리즘 적용 ==========

# 지도학습 모델 import - 입력에 대한 결과값이 있어 이를 학습하고, 새로운 데이터에 대한 결과 예측.
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# data 분리 
data = train.drop('Survived', axis=1).values
target = train['Survived'].values
# test_size: 분리 비율 설정. default=0.25
# stratify: 분리 기준이 될 데이터. default=None
# random_state: 랜덤 seed
# shuffle : split 해주기 전에 섞을 지 여부. default=True
x_train, x_valid, y_train, y_valid = train_test_split(data, target, test_size=0.4, stratify=target, random_state=0)

# 모델 적용 함수 
def ml_fit(model):
    model.fit(x_train, y_train)
    prediction = model.predict(x_valid)
    accuracy = accuracy_score(prediction, y_valid) # 전체 샘플 중 맞게 예측한 샘플 수의 비율
    print(model)
    print(f'총 {y_valid.shape[0]}명 중 {accuracy * 100:.3f}% 정확도로 생존을 맞춤')
    print('\n')
    return model

# 기본 설정으로만 테스트 
#model = ml_fit(RandomForestClassifier(n_estimators=100))
model = ml_fit(RandomForestClassifier(n_estimators=50, criterion="entropy", max_depth=5, oob_score=True, random_state=10))
model = ml_fit(LogisticRegression(solver='lbfgs'))
model = ml_fit(SVC(gamma='scale'))
model = ml_fit(KNeighborsClassifier())
model = ml_fit(GaussianNB())
model = ml_fit(DecisionTreeClassifier())

총 357명 중 82.913% 정확도로 생존을 맞춤


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=True, random_state=10, verbose=0,
                       warm_start=False)
총 357명 중 82.913% 정확도로 생존을 맞춤


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
총 357명 중 79.832% 정확도로 생존을 맞춤


SVC(C=1.0, break_ties=False, cache_siz